In [1]:
import requests
from bs4 import BeautifulSoup

In [2]:
# with~ 자동으로 closed 시켜준다
word_list= []
with open('./word.txt','r', encoding='UTF-8') as file_data:
    for line in file_data:
        word_list.append(line.strip())

In [3]:
def get_word_dic(lis, s_print=False):

    result_list = []
    
    for li in lis:
        result_dic = {}
        li_a = li.find('a','t_blue1')
        if li_a.find('sup') != None:
            r_seq = li_a.find('sup').text.strip()#순번
            result_dic['r_seq']= r_seq
            li_a.find('sup').decompose()

#         li.find('a','t_blue1').find('sup').decompose()#순번 삭제
        r_word = li.find('a','t_blue1').text.strip()#검색
        result_dic['r_word']=r_word


        r_link = 'http://stdict.korean.go.kr'+li.find('a','t_blue1')['href']#링크 가져오는방법
        result_dic['r_link']=r_link

        r_chi = li.find('span','t_gray hanja_font').text.strip()#한자
        result_dic['r_chi']=r_chi   

        r_des = li.find('font','dataLine').text.strip()#설명
        result_dic['r_des']=r_des   

        li_dt = li.find('dt')#dt가져오기
        li_dt.find('a','t_blue1').decompose()#a삭제
        li_dt.find('span').decompose()#span 삭제
        li_dt.find('span').decompose()#span 삭제
        r_pos = li_dt.text.strip()
        result_dic['r_pos']=r_pos

        result_list.append(result_dic)

    if s_print:
        for result in result_list:
            print(result['r_word'], end='')
            if 'r_seq' in result.keys(): 
                print(result['r_seq'], end='')                    
            print(result['r_chi'], end='')
            print(result['r_des'], end='')
            print(result['r_pos'] )
            print('    ',result['r_link'])
            print('--------------------------')
    
    return result_list

In [4]:
r_list = []

for word in word_list:

    url = 'https://stdict.korean.go.kr/search/searchResult.do?\
    pageSize=10&searchKeyword={}'.format(word)
    res = requests.get(url)
    soup = BeautifulSoup(res.content, 'html.parser')

    ul = soup.find('ul', 'result')
    # print(ul)

    if ul != None:
        lis = ul.find_all('li')
        r_list = r_list + get_word_dic(lis, False)
    else:
        print('검색결과를 못찾았습니다.')

In [5]:
from pymongo import MongoClient

In [6]:
client = MongoClient('localhost', 27017)

In [7]:
db = client['kor_dic_db2']

In [8]:
kor_dic_coll = db['kor_dic_coll2']

In [9]:
kor_dic_coll.insert_many(r_list)